In [2]:
import numpy as np # linear algebra
import pandas as pd

In [3]:
pd.pandas.set_option('display.max_columns',None)

In [4]:
from datetime import datetime

In [5]:
df = pd.read_csv('incident_event_log.csv')

In [6]:
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,Created by 6,29/2/2016 01:23,Updated by 21,29/2/2016 01:23,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,?,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,Created by 6,29/2/2016 01:23,Updated by 642,29/2/2016 08:53,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,?,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,Created by 6,29/2/2016 01:23,Updated by 804,29/2/2016 11:29,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,?,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,Created by 6,29/2/2016 01:23,Updated by 908,5/3/2016 12:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,?,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,Created by 171,29/2/2016 04:57,Updated by 746,29/2/2016 04:57,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [8]:
df['impact'].value_counts()

2 - Medium    113691
3 - Low         3418
1 - High        2889
Name: impact, dtype: int64

In [39]:
df['incident_state'].unique()

array(['New', 'Resolved', 'Closed', 'Active', 'Awaiting User Info',
       'Awaiting Problem', 'Awaiting Vendor', 'Awaiting Evidence', '-100'],
      dtype=object)

In [40]:
df.shape

(119998, 36)

In [41]:
columns = df.columns

In [42]:
df.replace('?', np.nan, inplace=True)


In [43]:
df.isna().sum()

number                          0
incident_state                  0
active                          0
reassignment_count              0
reopen_count                    0
sys_mod_count                   0
made_sla                        0
caller_id                      29
opened_by                    4835
opened_at                       0
sys_created_by              42354
sys_created_at              42354
sys_updated_by                  0
sys_updated_at                  0
contact_type                    0
location                       76
category                       64
subcategory                    97
u_symptom                   28271
cmdb_ci                    119562
impact                          0
urgency                         0
priority                        0
assignment_group            14213
assigned_to                 23030
knowledge                       0
u_priority_confirmation         0
notify                          0
problem_id                 118061
rfc           

In [44]:
columns_to_change_to_datetime = ['closed_at', 'opened_at' ]

for i in columns_to_change_to_datetime:
    df[i] = df[i].astype('datetime64[ns]')

In [45]:
df.dtypes

number                             object
incident_state                     object
active                               bool
reassignment_count                  int64
reopen_count                        int64
sys_mod_count                       int64
made_sla                             bool
caller_id                          object
opened_by                          object
opened_at                  datetime64[ns]
sys_created_by                     object
sys_created_at                     object
sys_updated_by                     object
sys_updated_at                     object
contact_type                       object
location                           object
category                           object
subcategory                        object
u_symptom                          object
cmdb_ci                            object
impact                             object
urgency                            object
priority                           object
assignment_group                  

In [46]:
df['opened_at']=df['opened_at'].dt.strftime("%d /%m /%y")

In [47]:
df[['opened_at']]

,opened_at
0,29 /02 /16
1,29 /02 /16
2,29 /02 /16
3,29 /02 /16
4,29 /02 /16
...,...
119993,13 /05 /16
119994,13 /05 /16
119995,13 /05 /16
119996,13 /05 /16


In [48]:
# life of a ticket in seconds
df['ticket_life'] = ((df['closed_at'] - df['opened_at']).dt.total_seconds())

C:\Users\user\anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1342: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(


TypeError: unsupported operand type(s) for -: 'Timestamp' and 'str'

In [ ]:
# tickets opened after closing
# Filtering out the rows where the opened_at > closed_at

negative_life = df[df.ticket_life<0 ]
negative_life.shape

In [ ]:
perc = negative_life.shape[0]/df.shape[0]*100
perc

In [ ]:
df.head()

In [ ]:
negative_life
.head()

In [ ]:
# tickets with negative life
negative_life[negative_life['reopen_count']<1].head()

In [ ]:
# total tickets with negative life and reopen count = 0
negative_life[negative_life['reopen_count']<1].shape

In [ ]:
negative_life[negative_life.reassignment_count > 1].shape

In [ ]:
df[df.reassignment_count<1].shape

In [ ]:
len(df.number)

In [ ]:
df.number.nunique()

In [ ]:
df.groupby(by='number').get_group(df.number[0])

In [ ]:
df = df[df.ticket_life>0]

In [ ]:
df.shape

In [ ]:
print('percentage of missing values'.upper())
df.isna().sum()/df.shape[0]*100

In [ ]:
drop = ['caused_by', 'rfc', 'vendor','cmdb_ci','problem_id']
df.drop(drop,axis=1,inplace=True)

In [ ]:
df.shape


In [ ]:
temp = df[['opened_at', 'sys_created_at']]
temp = temp.dropna().astype('datetime64[ns]')

In [ ]:
temp['gap'] = temp.sys_created_at - temp.opened_at

In [ ]:
import datetime
temp_mean = temp[temp.gap > datetime.timedelta(hours=0,minutes=0,seconds=0)].gap.mean()
temp_mean

In [ ]:
# df['sys_created_at'].fillna(value=(df.opened_at+temp_mean)).mean()
from datetime import timedelta

df['sys_created + temp_mean'] = df.opened_at + timedelta(minutes = 9, seconds=52.656746788)

In [ ]:
df.sys_created_at.fillna(value = df['sys_created + temp_mean'],inplace=True)

In [ ]:
temp2 = df[['opened_at', 'resolved_at']]

In [ ]:
temp2a = temp2.dropna()

In [ ]:
temp2a.resolved_at = temp2a.resolved_at.astype('datetime64[ns]')

In [ ]:
temp2a['new'] = temp2a.resolved_at - temp2a.opened_at

In [ ]:
temp2a.new.mean()

In [ ]:
import datetime
df['new2'] = df['opened_at'] + datetime.timedelta(days=5,hours = 5, minutes=34, seconds=59.468314879)

In [ ]:
df['resolved_at'].fillna(value = df.new2,inplace=True)

In [ ]:
df.isna().sum()


In [ ]:
# replacing with mode
df['closed_code'].fillna(df['closed_code'].mode()[0],inplace=True)
df['resolved_by'].fillna(df['resolved_by'].mode()[0],inplace=True)

In [ ]:
# creating a neww class for missing values  for caller id
df['caller_id'].fillna(value='caller 4340',inplace=True)

# creating a neww class for missing values for opened by
df['opened_by'].fillna(value = 'Opened by 4341', inplace=True)

# creating a neww class for missing values for sys created by
df['sys_created_by'].fillna(value='Created by 4342', inplace=True)

# creating a neww class for missing values for location
df['location'].fillna('Location 4343', inplace=True)

# creating a neww class for missing values for category
df['category'].fillna('Category 4344',inplace=True)

# creating a neww class for missing values for subcategory
df['subcategory'].fillna(value='Subcategory 4345',inplace=True)

# creating a neww class for missing values for symptoms
df['u_symptom'].fillna(value='Symptom 4346', inplace=True)

# creating a neww class for missing values for assignment group
df['assignment_group'].fillna(value='Group 4347',inplace=True)

# creating a neww class for missing values for resolver
df['assigned_to'].fillna(value='Resolver 4348',inplace=True)

# creating a neww class for missing values for problem id
# df['problem_id'].fillna(value='Promlem ID 4349',inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df1 = pd.read_csv('incident_event_log.csv')

In [ ]:
df1['assigned_to'].unique()

In [ ]:
df1['caller_id'].unique()

In [ ]:
df1['caller_id'].value_counts()

In [ ]:
df.shape()

In [ ]:
df.to_csv("INCIDENT MANAGEMENT.csv")